<a href="https://colab.research.google.com/github/Chthanh/Recommender-System/blob/main/K-Means%20Clustering%20Collaborative%20Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import numpy as np
import pandas as pd
from functools import reduce

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def euclidean_dist(x,y):
  """Hàm tính khoảng cách Euclidean giữa hai vector x và y"""
  dist = 0
  mutual_index = [i for i in range(len(x)) if not np.isnan(x[i]) and not np.isnan(y[i])]
  if len(mutual_index) != 0:
    for i in mutual_index:
      dist += (x[i] - y[i])**2
    return (np.sqrt(dist)/len(mutual_index))
  else:
    return 0

def pearson_corr(user_i, user_j):
  # Tìm index mà user_i và user_j đều được rate bởi tất cả các user
  mutual_index = [u for u in user_i.index if not np.isnan(user_i[u]) and not np.isnan(user_j[u])]

  dividend = 0
  i_divisor = 0
  j_divisor = 0

  # Tính độ tương đồng cosine
  for index in mutual_index:
    dividend += user_i[index] * user_j[index]
    i_divisor += pow(user_i[index], 2)
    j_divisor += pow(user_j[index], 2)

  divisor = np.sqrt(i_divisor) * np.sqrt(j_divisor) # Mẫu số của cosine

  if divisor != 0:
    return round(dividend / divisor, 5) # trả về độ tương đồng nếu mẫu số != 0

  return 0 # trả về 0 nếu mẫu số = 0

def add_vector(i, j):
  """Hàm cộng hai vector i và j"""
  return [np.nansum([i[k],j[k]]) for k in range(len(j))]

In [ ]:
def generate_centroids(k, data):
  """Chọn k users ngẫu nhiên của tập data làm centroids"""
  return data.sample(k, axis=1,  random_state = 12)

def assign_cluster(user, k, centroids):
  """Gán user vào cụm mà khoảng cách của user đến centroid của cụm đó có khoảng
  cách nhỏ nhất so với khoảng cách của user đến các centroids của các cụm khác
  @params:
    user: một user trong tập dữ liệu
    centroids: tập các centroids của các cụm
  @return:
    trả về cụm của user đó"""
  return min(range(k), key= lambda i: euclidean_dist(user.tolist(), centroids.iloc[:,i].tolist()))

def k_means(data, k):
  """
    Hàm nhóm các user thành k cụm
  """
  max_iter = 5
  iter = 0

  best_dist = np.inf # khởi tại khoảng cách ban đầu là vô định
  centroids = generate_centroids(k, data) # khởi tạo k centroids ngẫu nhiên từ tập data

  # Khởi tạo dict để lưu user và cụm tương ứng
  clusters = {}
  while iter < max_iter:
    # gán user vào cụm thứ k dựa trên khoảng cách nhỏ nhất giữa user đến các trung tâm cụm
    for user in data.columns:
      clusters[user] = assign_cluster(data[user], k, centroids)
    # Khởi tạo biến new_dist để lưu tổng khoảng cách mới
    new_dist = 0
    # Lặp qua từng assign_cluster(user: tên cụm tương ứng)
    for key, value in clusters.items():
      # Tính tổng khoảng cách của các user trong cụm đến centroid của cụm
      new_dist += euclidean_dist(data[key].tolist(), centroids.iloc[:,value].tolist())
    # Nếu tổng khoảng cách mới nhỏ hơn tổng khoảng cách của vòng lặp trước
    # thì cập nhật dist
    if new_dist < best_dist:
      best_dist = new_dist
      new_dist = 0
    else: # ngược lại trả về clusters: {user: cụm tương ứng} và centroids
      return clusters,  centroids
    # Cập nhật lại centroid của các cụm
    centroids = update_centroids(data, k, clusters)
    iter += 1

def update_centroids(data, k, clusters):
  """
  Hàm cập nhật lại centroid của các cụm
  """
  centroids = {}
  # Lặp qua k cụm để tạo các centroids mới
  for cen in range(k):
    # Tìm tất cả các thành viên của cụm này
    members = [data[key].tolist() for key, value in clusters.items() if value == cen]
    if members:
      l = [i/len(members) for i in reduce(add_vector, members)]
      centroids[cen] = l
  return pd.DataFrame.from_dict(centroids)

In [ ]:
def clustered_user_data(data, user_cluster, target_user):
  """Hàm trả về dataframe của cụm chứa target_user
  @params:
    data: data ban đầu
    user_cluster: dataframe gồm 2 cột: user và cụm tương ứng
    target_user: user mục tiêu
  @return: dataframe rating của user với các user của cụm có chứa target_user"""
  # Lấy tên cụm của target_user
  c = user_cluster[user_cluster['user'] == target_user]['cluster'].values
  # lấy tên tất cả các user trong cụm c
  user_list = list(user_cluster[user_cluster['cluster'] == c[0]]['user'])
  # Truy xuất lại dữ liệu ban đầu chỉ gồm những user thuộc về cụm c
  return data[user_list]

In [ ]:
def user_based_rec(data, user_cluster, target_user, threshold=0.2):
  """@param:
    data: dataframe user-item ban đầu
    target_user: user mục tiêu
    user_cluster: dataframe gồm 2 cột: item và cụm tương ứng
    @return: số điểm rating của target_user cho các item mà target_user chưa đánh giá
  """

  # Những item mà user mục tiêu đã đánh giá
  target_user_rated = data[target_user].dropna()
  target_user_rated_df = target_user_rated.reset_index().rename(columns={target_user:'rating', 'index':'item'}) # chuyển vị, reset lại index và đổi tên cột

  # Những item mà user mục tiêu chưa đánh giá
  target_user_unrated = data.drop(target_user_rated_df['item'], axis=0, errors='ignore')

  # Khởi tạo dictionary to lưu những item chưa được đánh giá và số điểm dự đoán tương ứng
  rating_prediction ={}

  # Truy xuất cụm data có chứa target user
  clustered_data = clustered_user_data(data, user_cluster, target_user)


  # Chuẩn hóa dữ liệu
  normalized_df = clustered_data.subtract(clustered_data.mean(axis = 0), axis='columns')
  mean = clustered_data.mean(axis = 0)
  # Lặp qua từng item trong tập item chưa được đánh giá
  for picked_item in target_user_unrated.index:
    # Khởi tạo tử số và mẫu số cho dự đoán rating
    nominator = 0
    denominator = 0
    # Chạy qua từng user trong cụm data
    for user in clustered_data:
      if user != target_user:
        # Tính hệ số cosine hiệu chỉnh giữa user và từng target_user
        similarity= pearson_corr(normalized_df[user], normalized_df[target_user])

        # rating của user cho item chưa được đánh giá
        rating = normalized_df[normalized_df.index==picked_item][user].values[0]
        if (pd.isna(rating) == False)& (similarity > threshold):
          nominator += rating * similarity
          denominator += similarity
    if(denominator != 0):
      rating_prediction[picked_item] = mean[target_user] + nominator/denominator
    else:
      rating_prediction[picked_item] = 0

  # Bước 7: Chuyển dictionary thành dataframe và sắp xếp dữ liệu theo prediction_score
  pred_score = (pd.DataFrame(rating_prediction.items(), columns=['item', 'pred_score'])
                      .sort_values(by='pred_score', ascending=False))
    # Trả về dataframe gồm item mà target_user chưa đánh giá cùng với số điểm rating dự đoán
  return pred_score

In [ ]:
# Load the dataset
data = pd.read_excel('data_01.xls')
data.rename(columns={"Unnamed: 0": "userId"}, inplace=True)
data.set_index('userId', inplace = True)
data.head()

,11: Star Wars: Episode IV - A New Hope (1977),12: Finding Nemo (2003),13: Forrest Gump (1994),14: American Beauty (1999),22: Pirates of the Caribbean: The Curse of the Black Pearl (2003),24: Kill Bill: Vol. 1 (2003),38: Eternal Sunshine of the Spotless Mind (2004),63: Twelve Monkeys (a.k.a. 12 Monkeys) (1995),77: Memento (2000),85: Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),...,8467: Dumb & Dumber (1994),8587: The Lion King (1994),9331: Clear and Present Danger (1994),9741: Unbreakable (2000),9802: The Rock (1996),9806: The Incredibles (2004),10020: Beauty and the Beast (1991),36657: X-Men (2000),36658: X2: X-Men United (2003),36955: True Lies (1994)
userId,,,,,,,,,,,,,,,,,,,,,
1648,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,5.0,3.5,3.0,NaN,3.5,NaN
5136,4.5,5.0,5.0,4.0,5.0,5.0,5.0,3.0,NaN,5.0,...,1.0,5.0,NaN,NaN,NaN,5.0,5.0,4.5,4.0,NaN
918,5.0,5.0,4.5,NaN,3.0,NaN,5.0,NaN,5.0,NaN,...,NaN,5.0,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN
2824,4.5,NaN,5.0,NaN,4.5,4.0,NaN,NaN,5.0,NaN,...,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3867,4.0,4.0,4.5,NaN,4.0,3.0,NaN,NaN,NaN,4.5,...,1.0,4.0,NaN,NaN,NaN,3.0,4.0,4.0,3.5,3.0


In [ ]:
data = data.T

In [ ]:
userID = 89
n_cluster = 3
data = data.T
u_cluster, cens = k_means(data, n_cluster)

u_cluster_df = pd.DataFrame(u_cluster.items(), columns=['user', 'cluster'])

for i in range(n_cluster):
  n_items = u_cluster_df[u_cluster_df['cluster'] == i].shape[0]
  print(f'Số lượng item trong cluster {i}: {n_items}')

user_based_rec(data, u_cluster_df, userID).head(10)

Số lượng item trong cluster 0: 5
Số lượng item trong cluster 1: 5
Số lượng item trong cluster 2: 15


,item,pred_score
4,85: Raiders of the Lost Ark (Indiana Jones and...,5.110169
33,807: Seven (a.k.a. Se7en) (1995),4.911888
3,77: Memento (2000),4.855387
5,98: Gladiator (2000),4.793499
23,568: Apollo 13 (1995),4.784740
10,134: O Brother Where Art Thou? (2000),4.781725
20,424: Schindler's List (1993),4.740745
30,745: The Sixth Sense (1999),4.738022
26,629: The Usual Suspects (1995),4.715018
9,122: The Lord of the Rings: The Return of the ...,4.712366
